In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import to_datetime, read_sql_query
from enum import Enum



In [5]:
class Period(Enum):
    date = '2021-10-01'
    wb=''

In [16]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query_presenceQty = f'''
SELECT 
    a.*, COUNT(*) AS qte_pres_by_creator
FROM
    (SELECT 
        dgs.id AS id_session,
            topic,
            date,
            dgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username
    FROM
        caris_db.dream_group_session dgs
    LEFT JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
    LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
GROUP BY a.created_by
'''

query_beneficiariesQty = f'''
SELECT *, COUNT(*)AS qte_by_creator FROM
    (SELECT 
        dgs.id AS id_session,
            topic,
            date,
            dga.id_patient AS id_patient,
            dgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username
    FROM
        caris_db.dream_group_session dgs
    INNER JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
        LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    

    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
            group by a.id_patient
'''

pres_byAgent= read_sql_query(query_presenceQty, engine, parse_dates=True)
ben_byAgent = read_sql_query(query_beneficiariesQty, engine, parse_dates=True)


# close the pool of connection
engine.dispose()


True

In [18]:
pres_byAgent
ben_byAgent

,id_session,topic,date,created_by,commune,departement,hub,email,username,qte_pres_by_creator
0,61533,8,2021-10-04,929,Pétionville,Ouest,College Saint Ignace d'Antioche,johane.jules@carisfoundationintl.org,johanejules,16
1,62839,12,2021-10-01,1553,Cap-Haïtien,Nord,Semi-Lycee Anacaona de Fort Saint-Michel,byronfabrice9@gmail.com,fbyron,512
2,63745,1,2021-11-06,1554,Tabarre,Ouest,Institution mixte Lumière du Monde\t,billyjolicoeur95@gmail.com,billyjolicoeur,291
3,64275,1,2021-11-06,1575,Pétionville,Ouest,Institution Mixte de l'Union,elionafredly7@gmail.com,FredlyEliona,132
4,63652,1,2021-10-30,1587,Saint-Marc,Artibonite,College Florvil Hypolite,blonnichel@yahoo.fr,dchelot,912
5,61532,8,2021-10-02,1635,Pétionville,Ouest,Gladys la maison des petits,gerald.poteau@carisfoundationintl.org,pggd1234,154
6,64018,1,2021-11-06,1636,Verrettes,Artibonite,Ecole Nationale Charlemagne Peralte/Verrettes,diegoarmando.dorlean@carisfoundationintl.org,Armando,1535
7,63608,1,2021-10-24,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,2346
8,63502,1,2021-10-23,1679,Dessalines,Artibonite,Ecole Fondamentale d'Application,francenelstjean.saed@yahoo.com,France14,2036
9,63984,1,2021-11-06,1883,Verrettes,Artibonite,Institution Mixte Karl Max/Desarmes,roselord.despeignes@carisfoundationintl.org,despeignes,566


,id_session,topic,date,id_patient,created_by,commune,departement,hub,email,username,qte_by_creator
0,64162,1,2021-11-06,76262,1883,Verrettes,Artibonite,Ecole Mixte Benito Merilus/Desarmes,roselord.despeignes@carisfoundationintl.org,despeignes,18
1,64047,1,2021-11-06,77949,1883,Verrettes,Artibonite,Institution Mixte Karl Max/Desarmes,roselord.despeignes@carisfoundationintl.org,despeignes,17
2,63608,1,2021-10-24,79096,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,17
3,63608,1,2021-10-24,79948,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,17
4,63608,1,2021-10-24,79949,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,17
...,...,...,...,...,...,...,...,...,...,...,...
555,64275,1,2021-11-06,128996,1575,Pétionville,Ouest,Institution Mixte de l'Union,elionafredly7@gmail.com,FredlyEliona,12
556,64275,1,2021-11-06,128997,1575,Pétionville,Ouest,Institution Mixte de l'Union,elionafredly7@gmail.com,FredlyEliona,12
557,64275,1,2021-11-06,128998,1575,Pétionville,Ouest,Institution Mixte de l'Union,elionafredly7@gmail.com,FredlyEliona,12
558,64275,1,2021-11-06,128999,1575,Pétionville,Ouest,Institution Mixte de l'Union,elionafredly7@gmail.com,FredlyEliona,12
